In [3108]:
import sys
import nltk
import sklearn
import pandas
import numpy

import pandas as pd
import numpy as np
df = pd.read_csv('Bug Reports Dataset.csv')
print (df.head(5))

   Bug ID Product Component          Assignee    Status Classification  \
0  528905     JDT      Core    jdt-core-inbox  RESOLVED        Eclipse   
1  530231     JDT      Core    jdt-core-inbox  RESOLVED        Eclipse   
2  530758     JDT      Core          jarthana  VERIFIED        Eclipse   
3  531990     JDT      Core    jdt-core-inbox  VERIFIED        Eclipse   
4  532137     JDT      Core  register.eclipse  VERIFIED        Eclipse   

  Priority            Opened  \
0       P1  12/18/2017 11:46   
1       P1    1/24/2018 5:16   
2       P1    2/5/2018 20:38   
3       P1     3/5/2018 0:49   
4       P1    3/7/2018 16:17   

                                             Summary       Category  
0  JDT UI Gerrit failing with "invalid location f...  Configuration  
1  Compilation failure in M20180123-0400 in jdt.c...  Configuration  
2                    Build failure in I20180205-2000  Configuration  
3                    Build failure in I20180304-2000  Configuration  
4  Todays up

In [3109]:
classes = df['Priority']
print(classes.value_counts())

P3    925
P2    539
P1    294
P5    225
P4    155
Name: Priority, dtype: int64


In [3110]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform (classes)
print(classes[:8])
print(Y[:8])
priority = df['Priority']
print(priority[:10])

0    P1
1    P1
2    P1
3    P1
4    P1
5    P1
6    P1
7    P1
Name: Priority, dtype: object
[0 0 0 0 0 0 0 0]
0    P1
1    P1
2    P1
3    P1
4    P1
5    P1
6    P1
7    P1
8    P1
9    P1
Name: Priority, dtype: object


In [3111]:
# Categorical boolean mask
categorical_feature_mask = df.dtypes == object

In [3112]:
categorical_feature_mask

Bug ID            False
Product            True
Component          True
Assignee           True
Status             True
Classification     True
Priority           True
Opened             True
Summary            True
Category           True
dtype: bool

In [3113]:
# filter categorical columns using mask and turn it into a list
categorical_cols = df.columns[categorical_feature_mask].tolist()
categorical_cols

['Product',
 'Component',
 'Assignee',
 'Status',
 'Classification',
 'Priority',
 'Opened',
 'Summary',
 'Category']

In [3114]:
pri_col = []
for i in categorical_cols:
    if i != 'Summary':
        pri_col.append(i)
print(pri_col)

['Product', 'Component', 'Assignee', 'Status', 'Classification', 'Priority', 'Opened', 'Category']


In [3115]:
le = LabelEncoder()
df[pri_col] = df[pri_col].apply(lambda col: le.fit_transform(col))
df[pri_col].head(10)

,Product,Component,Assignee,Status,Classification,Priority,Opened,Category
0,44,41,144,4,4,0,475,0
1,44,41,144,4,4,0,82,0
2,44,41,141,6,4,0,717,0
3,44,41,144,6,4,0,931,0
4,44,41,273,6,4,0,948,0
5,44,41,141,6,4,0,1367,0
6,22,41,7,1,8,0,105,0
7,14,272,227,2,9,0,1407,0
8,14,98,227,5,9,0,149,0
9,14,116,227,2,9,0,1610,0


In [3116]:
df.Priority.value_counts()

2    925
1    539
0    294
4    225
3    155
Name: Priority, dtype: int64

In [3117]:
from sklearn.utils import resample
df_minority0 = df[df.Priority==0]
df_minority1 = df[df.Priority==1]
df_minority2 = df[df.Priority==2]
df_minority3 = df[df.Priority==3]
df_minority4 = df[df.Priority==4]

In [3118]:
df_minority_upsampled0 = resample(df_minority0, 
                                 replace=True,     # sample with replacement
                                 n_samples=600)    # to match majority class
df_minority_upsampled1 = resample(df_minority1, 
                                 replace=True,     # sample with replacement
                                 n_samples=400)    # to match majority class
df_minority_upsampled2 = resample(df_minority2, 
                                 replace=True,     # sample with replacement
                                 n_samples=250)    # to match majority class
df_minority_upsampled3 = resample(df_minority3, 
                                 replace=True,     # sample with replacement
                                 n_samples=550)    # to match majority class
df_minority_upsampled4 = resample(df_minority4, 
                                 replace=True,     # sample with replacement
                                 n_samples=400)

In [3119]:
df = pd.concat([df, df_minority_upsampled0, df_minority_upsampled1, df_minority_upsampled2, df_minority_upsampled3, df_minority_upsampled4])

In [3120]:
df.Priority.value_counts()

2    1175
1     939
0     894
3     705
4     625
Name: Priority, dtype: int64

In [3121]:
X  = df[['Product','Classification','Component', 'Category', 'Assignee','Status']]

In [3122]:
print(X.shape)

(4338, 6)


In [3123]:
Y = df['Priority']

In [3124]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [3125]:
print(Y.value_counts())

2    1175
1     939
0     894
3     705
4     625
Name: Priority, dtype: int64


In [3126]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [3127]:
predictions = text_classifier.predict(X_test)

In [3128]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

[[185   4   6   3   0]
 [ 10 172   8   2   1]
 [ 10  19 182   9   6]
 [  1   1   0 127  10]
 [  4   0   0   6 102]]
              precision    recall  f1-score   support

           0       0.88      0.93      0.91       198
           1       0.88      0.89      0.88       193
           2       0.93      0.81      0.86       226
           3       0.86      0.91      0.89       139
           4       0.86      0.91      0.88       112

    accuracy                           0.88       868
   macro avg       0.88      0.89      0.88       868
weighted avg       0.89      0.88      0.88       868

0.8847926267281107


In [3129]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

In [3130]:
print(confusion_matrix(y_test,y_pred))  
print(metrics.accuracy_score(y_test, y_pred))
print(classification_report(y_test,y_pred))  

[[ 50  75  27  19  27]
 [ 24 136  16   1  16]
 [ 22  24 100  47  33]
 [ 33  15  33  44  14]
 [ 29  12  11  14  46]]
0.43317972350230416
              precision    recall  f1-score   support

           0       0.32      0.25      0.28       198
           1       0.52      0.70      0.60       193
           2       0.53      0.44      0.48       226
           3       0.35      0.32      0.33       139
           4       0.34      0.41      0.37       112

    accuracy                           0.43       868
   macro avg       0.41      0.43      0.41       868
weighted avg       0.43      0.43      0.42       868



In [3135]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(solver='liblinear', random_state=0)
#fit the model
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)

In [3136]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred_test))  
print(metrics.accuracy_score(y_test, y_pred_test))
print(classification_report(y_test,y_pred_test))  

[[ 65  67  47   7  12]
 [ 36 126  22   1   8]
 [ 31  45 107  19  24]
 [ 28  15  62  24  10]
 [ 27  10  33   6  36]]
0.41244239631336405
              precision    recall  f1-score   support

           0       0.35      0.33      0.34       198
           1       0.48      0.65      0.55       193
           2       0.39      0.47      0.43       226
           3       0.42      0.17      0.24       139
           4       0.40      0.32      0.36       112

    accuracy                           0.41       868
   macro avg       0.41      0.39      0.38       868
weighted avg       0.41      0.41      0.40       868



In [3133]:
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(criterion = 'entropy').fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 
print("Accuracy:",metrics.accuracy_score(y_test, dtree_predictions))

Accuracy: 0.8732718894009217


In [3134]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,dtree_predictions))  
print(classification_report(y_test,dtree_predictions))  
print(accuracy_score(y_test, dtree_predictions))

[[186   3   5   3   1]
 [ 11 168  11   2   1]
 [ 10  20 170  16  10]
 [  1   1   0 132   5]
 [  4   0   0   6 102]]
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       198
           1       0.88      0.87      0.87       193
           2       0.91      0.75      0.83       226
           3       0.83      0.95      0.89       139
           4       0.86      0.91      0.88       112

    accuracy                           0.87       868
   macro avg       0.87      0.88      0.87       868
weighted avg       0.88      0.87      0.87       868

0.8732718894009217
